In [14]:
# %% [markdown]
# # Model Evaluation - ripenessVision
# 
# Evaluasi mendalam model yang sudah ditraining

# %%
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import cv2


In [15]:
# %%
# Setup
BASE_DIR = Path('..')
MODELS_DIR = BASE_DIR / 'models'
RESULTS_DIR = BASE_DIR / 'results'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

# %%
# Load model and data
model = keras.models.load_model(str(MODELS_DIR / 'best_model.h5'))

test_df = pd.read_csv(PROCESSED_DIR / 'test_metadata.csv')

with open(PROCESSED_DIR / 'data_splits.json', 'r') as f:
    split_info = json.load(f)

classes = split_info['classes']

In [19]:
# %%
# Load model and data dengan error handling
try:
    # Load model dengan compile=False untuk menghindari warning
    model = keras.models.load_model(
        str(MODELS_DIR / 'best_model.h5'), 
        compile=False  # TAMBAH INI
    )
    
    # Recompile model setelah load
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ Model loaded and recompiled successfully!")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Trying alternative loading method...")
    
    # Alternative loading method
    try:
        model = keras.models.load_model(str(MODELS_DIR / 'best_model.h5'))
        print("✅ Model loaded with default method!")
    except:
        print("❌ Still failed. Checking available models...")
        
        # List semua model yang available
        model_files = list(MODELS_DIR.glob('*.h5'))
        print("Available model files:")
        for model_file in model_files:
            print(f"  - {model_file.name}")
        
        # Load model lain jika best_model tidak ada
        if model_files:
            model = keras.models.load_model(str(model_files[0]))
            print(f"✅ Loaded alternative model: {model_files[0].name}")
        else:
            raise FileNotFoundError("No model files found!")

# Load test data
try:
    test_df = pd.read_csv(PROCESSED_DIR / 'test_metadata.csv')
    print(f"✅ Test data loaded: {len(test_df)} samples")
except:
    print("❌ test_metadata.csv not found, trying filtered version...")
    test_df = pd.read_csv(PROCESSED_DIR / 'test_metadata_filtered.csv')
    print(f"✅ Filtered test data loaded: {len(test_df)} samples")

# Load class information
try:
    with open(PROCESSED_DIR / 'data_splits.json', 'r') as f:
        split_info = json.load(f)
    classes = split_info['classes']
    print(f"✅ Classes loaded: {classes}")
except:
    print("❌ data_splits.json not found, extracting from dataframe...")
    classes = sorted(test_df['class'].unique().tolist())
    print(f"✅ Classes extracted from dataframe: {classes}")

# Verifikasi model dan data compatibility
print(f"\n📊 MODEL-DATA COMPATIBILITY CHECK:")
print(f"Model input shape: {model.input_shape}")
print(f"Model output shape: {model.output_shape}")
print(f"Number of classes in data: {len(classes)}")
print(f"Model output classes: {model.output_shape[1]}")

if model.output_shape[1] != len(classes):
    print("⚠️  WARNING: Model output dimension doesn't match number of classes!")
    print("This might cause issues during evaluation.")
else:
    print("✅ Model and data classes are compatible!")

✅ Model loaded and recompiled successfully!
✅ Test data loaded: 120 samples
✅ Classes loaded: ['tomato_unripe', 'tomato_ripe', 'tomato_overripe', 'mango_unripe', 'mango_ripe', 'mango_overripe', 'banana_unripe', 'banana_ripe', 'banana_overripe']

📊 MODEL-DATA COMPATIBILITY CHECK:
Model input shape: (None, 224, 224, 3)
Model output shape: (None, 9)
Number of classes in data: 9
Model output classes: 9
✅ Model and data classes are compatible!


In [20]:
# %%
# Create test data generator
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=classes
)


Found 120 validated image filenames belonging to 9 classes.


In [22]:
# %%
# Comprehensive evaluation dengan error handling yang diperbaiki
print("=== COMPREHENSIVE MODEL EVALUATION ===")

def safe_evaluate_model(model, test_df, classes, batch_size=16):
    """
    Evaluate model dengan safety mechanism - FIXED VERSION
    """
    print("🔍 Filtering test data for corrupt images...")
    
    # Filter test data untuk hanya include valid images
    valid_test_indices = []
    for idx, row in test_df.iterrows():
        try:
            with Image.open(row['file_path']) as img:
                img.verify()
            if os.path.getsize(row['file_path']) > 0:
                valid_test_indices.append(idx)
        except:
            print(f"Removing corrupt test image: {row['file_path']}")
            continue
    
    valid_test_df = test_df.loc[valid_test_indices]
    print(f"✅ Valid test images: {len(valid_test_df)}/{len(test_df)}")
    
    # CHECK: Pastikan kolom 'class' ada di DataFrame
    print("📋 Checking DataFrame columns...")
    print(f"Available columns: {valid_test_df.columns.tolist()}")
    
    # Jika kolom 'class' tidak ada, coba kolom alternatif
    if 'class' not in valid_test_df.columns:
        print("⚠️  Column 'class' not found. Looking for alternative columns...")
        
        # Coba kolom alternatif yang mungkin ada
        possible_class_columns = ['class', 'label', 'category', 'ripeness_class']
        class_column_found = None
        
        for col in possible_class_columns:
            if col in valid_test_df.columns:
                class_column_found = col
                print(f"✅ Found alternative class column: '{col}'")
                break
        
        if class_column_found is None:
            # Jika tidak ada kolom class, buat dari kombinasi fruit dan ripeness
            print("🔄 Creating 'class' column from fruit and ripeness...")
            if 'fruit' in valid_test_df.columns and 'ripeness' in valid_test_df.columns:
                valid_test_df['class'] = valid_test_df['fruit'] + '_' + valid_test_df['ripeness']
                class_column_found = 'class'
                print("✅ Created 'class' column successfully")
            else:
                raise KeyError("No class column found and cannot create one from available columns")
    else:
        class_column_found = 'class'
    
    # Create safe test generator dengan kolom yang benar
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    print(f"📊 Creating generator with class column: '{class_column_found}'")
    
    test_generator = test_datagen.flow_from_dataframe(
        valid_test_df,
        x_col='file_path',
        y_col=class_column_found,  # Gunakan kolom yang ditemukan
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        classes=classes
    )
    
    # Evaluate model
    print("📊 Evaluating model...")
    if len(valid_test_df) > 0:
        test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
    else:
        print("❌ No valid test images available for evaluation!")
        test_loss, test_accuracy = 0, 0
    
    return test_loss, test_accuracy, valid_test_df

# Jalankan safe evaluation
try:
    test_loss, test_accuracy, valid_test_df = safe_evaluate_model(model, test_df, classes)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    print("Trying alternative evaluation method...")

=== COMPREHENSIVE MODEL EVALUATION ===
🔍 Filtering test data for corrupt images...
Removing corrupt test image: ..\data\raw\mango\unripe\unripe_mango_high_quality_photo_86.jpg
Removing corrupt test image: ..\data\raw\tomato\overripe\overripe_tomato_high_quality_photo_75.jpg
Removing corrupt test image: ..\data\raw\banana\overripe\overripe_banana_high_quality_photo_58.jpg
Removing corrupt test image: ..\data\raw\tomato\overripe\overripe_tomato_high_quality_photo_20.jpg
Removing corrupt test image: ..\data\raw\mango\unripe\unripe_mango_high_quality_photo_30.jpg
Removing corrupt test image: ..\data\raw\mango\unripe\unripe_mango_high_quality_photo_23.jpg
Removing corrupt test image: ..\data\raw\tomato\overripe\overripe_tomato_high_quality_photo_31.jpg
Removing corrupt test image: ..\data\raw\banana\unripe\unripe_banana_high_quality_photo_0.jpg
Removing corrupt test image: ..\data\raw\mango\unripe\unripe_mango_high_quality_photo_62.jpg
Removing corrupt test image: ..\data\raw\mango\ripe\rip

In [ ]:
# %%
# Predictions
y_true = test_generator.classes
y_pred_proba = model.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)